In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
from utils import * 
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from training_audio_model import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=False)

data[:3]

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


,ipu_id,speaker,start,stop,text,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,dyad
0,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,transcr\AAOR
1,1,OR,5.14,5.825,mh ouais si tu veux,True,True,False,NaN,False,True,transcr\AAOR
2,2,OR,6.62,7.010,frog joke,True,False,False,NaN,False,False,transcr\AAOR


In [2]:

df = pd.DataFrame(data)

print("y de base")
y=create_y(df) # Création de la target
print(y.count(1))
print(y.count(0))


print("y yield at")
y=create_y_yield_at(df) # Création de la target
print(y.count(1))
print(y.count(0))


"""
print("y turn after")
y=create_y_turn_after(df) # Création de la target
print(y.count(1))
print(y.count(0))
print(len(y))
print(len(data))
"""

y de base
9131
7269
y yield at
3090
13310


'\nprint("y turn after")\ny=create_y_turn_after(df) # Création de la target\nprint(y.count(1))\nprint(y.count(0))\nprint(len(y))\nprint(len(data))\n'

# Importation des données audios

In [3]:

audio_files_path = 'paco-cheese/audio/2_channels/'
audio_segments = extract_audio_segments(data,audio_files_path)
X = np.array([extract_features(segment) for segment in audio_segments])


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1280
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1120
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=18

In [4]:
dataset = AudioDataset(X, y)
train_size=int(0.8*len(dataset))
X_train = dataset[:train_size]
y_train = y[:train_size]


X_test = dataset[train_size:]
y_test = y[train_size:]

# Divisez les données en ensembles d'entraînement et de test
#train_size = int(0.8 * len(dataset))
#test_size = len(dataset) - train_size
#train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader_audio = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
print(len(y))
print(len(y_test))

16400
3280


In [6]:
model=torch.load('modele/model_audio')
all_preds_audio,all_labels=predition_model_audio(model,test_loader_audio,device,proba=False)
f1 = f1_score(all_labels, all_preds_audio, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_audio)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')


Test F1 Score: 0.6974046347065446
Confusion Matrix:
[[1996  625]
 [ 417  242]]
Nombre d'éléments de classe 0 détectés : 1996 sur 2621
Nombre d'éléments de classe 1 détectés : 242 sur 659


In [7]:
#model = importation

all_preds_audio,all_labels_audio=predition_model_audio(model,test_loader_audio,device,proba=True)
print(all_preds_audio)
print(len(all_preds_audio))

[[ 0.35173398 -0.08440793]
 [ 0.24370825 -0.01410505]
 [ 0.30740234 -0.07149775]
 ...
 [ 0.6129222  -0.35087425]
 [ 0.38541636 -0.1861543 ]
 [ 1.442695   -1.3897967 ]]
3280


# Importation données texte

In [8]:
from training_text_model import *

seed = 42

data_text=create_sequences(data)
# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data_text, y, test_size=0.2, random_state=seed)

# Paramètres
model_name = 'camembert-base'
max_length = 256
batch_size = 16
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tokenizer pour CamemBERT
tokenizer = CamembertTokenizer.from_pretrained(model_name)


# Création des datasets et dataloaders pour le texte
train_dataset = TextDataset(X_train, y_train, tokenizer, max_length)
train_loader_text = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TextDataset(X_test, y_test, tokenizer, max_length)
test_loader_text = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## prediction modèle texte

In [9]:
all_preds_text = []
all_labels = []
model= torch.load('modele/camembert_epoch_3.bin')
model.eval()
all_preds_text,all_labels = prediction_model_text(model,test_loader_text,device,proba=False)
f1 = f1_score(all_labels, all_preds_text)#, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_text)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')

Test F1 Score: 0.3306982872200263
Confusion Matrix:
[[ 746 1926]
 [ 106  502]]
Nombre d'éléments de classe 0 détectés : 746 sur 2672
Nombre d'éléments de classe 1 détectés : 502 sur 608


# prédiction late fusion

In [10]:

model_text= torch.load('modele/camembert_epoch_1.bin')
model_text.eval()
all_preds_text_proba,all_labels = prediction_model_text(model_text,test_loader_text,device,proba=True)

#combined_preds_and = [pred_text and pred_audio for pred_text, pred_audio in zip(all_preds_text, all_preds_audio)]


In [11]:
model_audio=torch.load('modele/model_audio')
import torch
import numpy as np


all_preds_audio_proba,all_labels_audio=predition_model_audio(model_audio,test_loader_audio,device,proba=True)

In [12]:
print(all_preds_audio)
print(all_preds_text)


[[ 0.35173398 -0.08440793]
 [ 0.24370825 -0.01410505]
 [ 0.30740234 -0.07149775]
 ...
 [ 0.6129222  -0.35087425]
 [ 0.38541636 -0.1861543 ]
 [ 1.442695   -1.3897967 ]]
[0 0 1 ... 1 1 1]


In [13]:
import numpy as np
# Addition des tableaux de probabilités
sum_array = np.add(all_preds_audio_proba, all_preds_text_proba)

# Trouver la classe pour chaque liste (0 ou 1)
combined_preds = np.argmax(sum_array, axis=1)


In [14]:
f1 = f1_score(all_labels, combined_preds, average='weighted')
conf_matrix = confusion_matrix(all_labels, combined_preds)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')


total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')



Test F1 Score: 0.46661614590325806
Confusion Matrix:
[[ 936 1736]
 [ 149  459]]
Nombre d'éléments de classe 0 détectés : 936 sur 2672
Nombre d'éléments de classe 1 détectés : 459 sur 608
